# Init an instance

In [ ]:
import lamindb as ln
from laminci.db import setup_local_test_postgres
from pathlib import Path

We already set up a user account for "testuser1@lamin.ai" and chose handle `testuser1`.

In [ ]:
ln.setup.login("testuser1")  # CLI: lamin login testuser1

## Local database & storage

### SQLite

In [ ]:
ln.setup.init(storage="./mydata")  # CLI: lamin init --storage ./mydata

This automatically assigns an instance name that equals the name of the storage root along with a few other settings:

In [ ]:
ln.setup.settings.instance

In [ ]:
assert ln.setup.settings.instance.storage.is_cloud == False
assert ln.setup.settings.instance.owner == ln.setup.settings.user.handle
assert ln.setup.settings.instance.name == "mydata"
assert ln.setup.settings.storage.root.as_posix() == Path("mydata").resolve().as_posix()
assert ln.setup.settings.storage.cache_dir is None
assert (
    ln.setup.settings.instance.db
    == f"sqlite:///{Path('./mydata').resolve().as_posix()}/mydata.lndb"
)

If you want to register it on the hub at lamin.ai, call:
```
ln.setup.register()
```

### Postgres

In [ ]:
pgurl = setup_local_test_postgres()

A connection string for postgres looks like this:

In [ ]:
pgurl

Let us call init:

In [ ]:
ln.setup.init(storage="./mydatapg", db=pgurl)

In [ ]:
assert ln.setup.settings.instance.name == "pgtest"
assert ln.setup.settings.instance.storage.is_cloud == False
assert ln.setup.settings.instance.owner == ln.setup.settings.user.handle
assert ln.setup.settings.instance.dialect == "postgresql"
assert ln.setup.settings.instance.db == pgurl
assert (
    ln.setup.settings.instance.storage.root.as_posix()
    == Path("mydatapg").absolute().as_posix()
)
assert ln.setup.settings.instance.storage.cache_dir is None

!lamin delete pgtest
!docker stop pgtest && docker rm pgtest

### Custom instance name

In [ ]:
pgurl = setup_local_test_postgres()

Instead of having the instance name be auto-determined from `storage` or `db`, you can provide a custom name:

In [ ]:
ln.setup.init(
    storage="./mystorage", name="mydata2", db=pgurl
)  # CLI: lamin init --storage ./mystorage --name "mydata" --db {pgurl}

In [ ]:
assert ln.setup.settings.instance.name == "mydata2"
assert ln.setup.settings.instance.storage.is_cloud == False
assert ln.setup.settings.instance.owner == ln.setup.settings.user.handle
assert ln.setup.settings.instance.dialect == "postgresql"
assert ln.setup.settings.instance.db == pgurl
assert (
    ln.setup.settings.instance.storage.root.as_posix()
    == Path("mystorage").absolute().as_posix()
)
assert ln.setup.settings.instance.storage.cache_dir is None

!lamin delete mydata2
!docker stop pgtest && docker rm pgtest

## Configure with cloud storage

### AWS

You need to have access to AWS S3 via `awscli configure`.

Let us look at the special case of an sqlite instance:

In [ ]:
ln.setup.init(
    storage="s3://lndb-setup-ci"
)  # CLI: lamin init --storage "s3://lndb-setup-ci"

In [ ]:
ln.setup.settings.instance

In [ ]:
ln.setup.settings.instance._sqlite_file

In [ ]:
ln.setup.settings.instance._sqlite_file_local

In [ ]:
# test
assert ln.setup.settings.storage.is_cloud == True
assert str(ln.setup.settings.storage.root) == "s3://lndb-setup-ci/"
assert ln.setup.settings.storage.region == "us-east-1"
assert (
    str(ln.setup.settings.instance._sqlite_file)
    == "s3://lndb-setup-ci/lndb-setup-ci.lndb"
)

# do the same for an S3 bucket in Europe
ln.setup.init(storage="s3://lndb-setup-ci-eu-central-1", name="lndb-setup-ci-europe")
assert ln.setup.settings.storage.region == "eu-central-1"
assert ln.setup.settings.instance.name == "lndb-setup-ci-europe"
assert (
    str(ln.setup.settings.instance._sqlite_file)
    == "s3://lndb-setup-ci-eu-central-1/lndb-setup-ci-europe.lndb"
)
ln.setup.delete("lndb-setup-ci-europe")

### GCP

You need to authenticate for Google Clod.

* Either, set the environment variable `export GOOGLE_APPLICATION_CREDENTIALS=<HOME-DIR>/.lndb/<GOOGLE CLOUD PROJECT>.json`.
* Alternatively, if you set up the `gcloud` CLI, log in with `gcloud auth application-default login`.


In [ ]:
ln.setup.init(storage="gs://lndb-setup-ci-us")

In [ ]:
ln.setup.delete("lndb-setup-ci-us")

In [ ]:
# this should move up in this guide
pgurl = setup_local_test_postgres(name="pgtest-registered")
ln.setup.init(storage="s3://lndb-setup-ci", db=pgurl)
ln.setup.register()

## Re-initialize an existing instance

Assume we accidentally `init` an existing instance, it will be loaded:

In [ ]:
assert ln.setup.init(storage="mydata") == "migrate-unnecessary"

In [ ]:
assert type(ln.setup.settings.storage.id)